In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

1.15.0-rc1
2.2.4-tf


## Get an example dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 4s 0us/step


## Define a model

In [3]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation=keras.activations.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=keras.activations.softmax)
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

Checkpoint callback usage

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training  

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 992/1000 [============================>.] - ETA: 0s - loss: 1.1753 - acc: 0.6683
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 159us/sample - loss: 1.1672 - acc: 0.6710 - val_loss: 0.7522 - val_acc: 0.7500
Epoch 2/10
  32/1000 [..............................] - ETA: 0s - loss: 0.7201 - acc: 0.8125
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 84us/sample - loss: 0.4275 - acc: 0.8760 - val_loss: 0.5427 - val_acc: 0.8370
Epoch 3/10
  32/1000 [..............................] - ETA: 0s - loss: 0.2647 - acc: 0.9375
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 82us/sample - loss: 0.2890 - acc: 0.9170 - val_loss: 0.4600 - val_acc: 0.8520
Epoch 4/10
  32/1000 [..............................] - ETA: 0s - loss: 0.2850 - acc: 0.8438
Epoch 00004: saving model to training_1/cp.ckpt
1000/1000

In [5]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.3843 - acc: 0.0880
Untrained model, accuracy:  8.80%


In [6]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4117 - acc: 0.8640
Restored model, accuracy: 86.40%


## Checkpoint callback options

In [12]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [8]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [9]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc)) 

1000/1000 - 0s - loss: 0.4975 - acc: 0.8680
Restored model, accuracy: 86.80%


## Manually save weights

In [13]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4799 - acc: 0.8650
Restored model, accuracy: 86.50%


## Save the entire model

This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration 

<font color='red' size=5>Caution:</font>
Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from tf.train). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.

### As an HDFS file

In [14]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 92us/sample - loss: 1.1606 - acc: 0.6830
Epoch 2/5
1000/1000 [==============================] - 0s 47us/sample - loss: 0.4229 - acc: 0.8840
Epoch 3/5
1000/1000 [==============================] - 0s 47us/sample - loss: 0.2809 - acc: 0.9260
Epoch 4/5
1000/1000 [==============================] - 0s 49us/sample - loss: 0.2045 - acc: 0.9530
Epoch 5/5
1000/1000 [==============================] - 0s 49us/sample - loss: 0.1539 - acc: 0.9700


In [15]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4277 - acc: 0.8660
Restored model, accuracy: 86.60%


## As a saved_model

In [17]:
model = create_model()
model.fit(train_images, train_labels, epochs=3)

Train on 1000 samples
Epoch 1/3
1000/1000 [==============================] - 0s 97us/sample - loss: 1.1449 - acc: 0.6560
Epoch 2/3
1000/1000 [==============================] - 0s 51us/sample - loss: 0.4219 - acc: 0.8810
Epoch 3/3
1000/1000 [==============================] - 0s 48us/sample - loss: 0.2900 - acc: 0.9260


In [20]:
import time
saved_model_path = "./saved_models/" + str(int(time.time()))
tf.contrib.saved_model.save_keras_model(model, saved_model_path)
!ls {saved_model_path}

INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: ['eval']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tens

In [23]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4580 - acc: 0.8580
Restored model, accuracy: 85.80%
